In [1]:
from pathlib import Path
import numpy as np

# Plotting
import cmocean
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
color_map = cmocean.cm.ice

# Methods
from s4hci.models.psf_subtraction import S4
from s4hci.utils.pca import pca_psf_subtraction_gpu

from s4hci.utils.data_handling import save_as_fits
from applefy.utils.file_handling import load_adi_data

# Find all datasets

In [2]:
root_dir = Path("/fast/mbonse/NACO/30_data/")

In [3]:
list(root_dir.iterdir())

[PosixPath('/fast/mbonse/NACO/30_data/HD196544_131_087_C-0292_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD166348_128_1101_C-0092_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD38206_331_084_C-0396_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD39060_23_094_C-0149_D_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD22049_304_199_C-0065_C_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD199260_223_1101_C-0092_I_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/ELIA2-27_105_1101_C-0092_E_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD139664_221_1101_C-0092_I_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD207129_156_1101_C-0092_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD109832_146_1101_C-0092_F_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD197481_209_085_C-0675_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD138965_225_087_C-0292_A_sel.hdf5'),
 PosixPath('/fast/mbonse/NACO/30_data/HD188228_142_1101_C-0092_E_sel.hdf5'),
 PosixPat

In [4]:
test_dataset = list(root_dir.iterdir())[12]
test_dataset

PosixPath('/fast/mbonse/NACO/30_data/HD188228_142_1101_C-0092_E_sel.hdf5')

In [5]:
tmp_result_dir = Path("/fast/mbonse/NACO/70_results") / test_dataset.name[:-9]
tmp_result_dir

PosixPath('/fast/mbonse/NACO/70_results/HD188228_142_1101_C-0092_E')

In [6]:
tmp_result_dir.mkdir(exist_ok=True)

In [7]:
science_data, angles, raw_psf_template_data = load_adi_data(
    test_dataset,
    data_tag="object_hr_stacked_05",
    psf_template_tag="psf_template",
    para_tag="header_object_hr_stacked_05/PARANG")

# Background subtraction of the PSF template
psf_template_data = np.median(raw_psf_template_data, axis=0)

In [8]:
fwhm = 3.6
pixel_scale = 0.0271

# Run PCA

In [9]:
work_dir = tmp_result_dir / Path("PCA")
work_dir.mkdir(exist_ok=True)

In [10]:
mean_residual =  pca_psf_subtraction_gpu(
    images = science_data,
    angles = angles,
    pca_numbers = np.arange(500),
    device=0,
    approx_svd=np.min([science_data.shape[0]-200, 5000]),
    subsample_rotation_grid = 1,
    verbose=True,
    combine="mean")

Compute PCA basis ...[DONE]
Compute PCA residuals ...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 56.67it/s]


[DONE]


In [11]:
median_residual =  pca_psf_subtraction_gpu(
    images = science_data,
    angles = angles,
    pca_numbers = np.arange(500),
    device=0,
    approx_svd=np.min([science_data.shape[0]-200, 5000]),
    subsample_rotation_grid = 1,
    verbose=True,
    combine="median")

Compute PCA basis ...[DONE]
Compute PCA residuals ...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:25<00:00, 19.77it/s]


[DONE]


In [12]:
all_residuals = np.array([mean_residual, median_residual])

In [14]:
save_as_fits(all_residuals, work_dir / Path("pca_residuals.fits"), overwrite=True)